In [ ]:
import warnings

import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.units import UnitsWarning
from sklearn.neighbors import KNeighborsRegressor
from tqdm import tqdm

from plato.stars import classify_stars, quality_cuts
from plato.utils import accumulate_from_sources, get_abspath

In [ ]:
# process targets

# load targets
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UnitsWarning)
    targets = Table.read(get_abspath() + "data/raw/plato_targets.fits").to_pandas()

# make quality cuts
categorized_targets = quality_cuts(targets, max_error=0.2)

# add classification
categorized_targets = classify_stars(
    categorized_targets,
    include_galactic_quantities=True,
    include_probabilities=True,
)

# merge back in with targets
targets = targets.merge(
    categorized_targets[
        [col for col in categorized_targets.columns if col not in targets.columns]
        + ["dr2_source_id"]
    ],
    on="dr2_source_id",
    how="left",
)

# rename columns
targets.rename(
    columns={
        "SOURCE_ID": "gaiaID_DR3",
        "dr2_source_id": "gaiaID_DR2",
    },
    inplace=True,
)
for col in targets.columns:
    if col.endswith("_error"):
        targets.rename(columns={col: f"e_{col[:-6]}"}, inplace=True)

In [ ]:
# add xgboost metallicities from Andrae2023
total_rows = 174922161
chunksize = int(1e7)
xgboost_data = []

# load in chuncks
for chunk in tqdm(
    pd.read_csv(
        get_abspath() + "data/external/xgboost.csv",
        chunksize=chunksize,
        compression="gzip",
        usecols=["source_id", "mh_xgboost", "logg_xgboost"],
    ),
    total=total_rows // chunksize + 1,
    desc="Processing chunks: ",
):
    # Select only the targets
    filtered_chunk = chunk[chunk["source_id"].isin(targets["gaiaID_DR3"])]
    xgboost_data.append(filtered_chunk)

xgboost_data = pd.concat(xgboost_data).rename(columns={"source_id": "gaiaID_DR3"})
xgboost_data = xgboost_data.assign(
    mh_xgboost_lower=np.nan,
    mh_xgboost_upper=np.nan,
    logg_xgboost_lower=np.nan,
    logg_xgboost_upper=np.nan,
)
targets = targets.merge(xgboost_data, on="gaiaID_DR3", how="left")

In [ ]:
# add [Fe/H] metallcity column, filled in following priority: gspspec,
# xgboost (use next possible source if previous is NaN)
targets = accumulate_from_sources(
    targets,
    "[Fe/H]",
    source_columns=["mh_gspspec", "mh_xgboost"],
    drop_sources=True,
)

# add logg column, filled in following priority: gspspec, gspphot, xgboost
targets = accumulate_from_sources(
    targets,
    "logg",
    source_columns=["logg_gspspec", "logg_xgboost"],
    drop_sources=True,
)


# rename alphafe_gspspec column (and errors) to [alpha/Fe]
targets.rename(
    columns={
        "alphafe_gspspec": "[alpha/Fe]",
        "alphafe_gspspec_lower": "e_[alpha/Fe]_lower",
        "alphafe_gspspec_upper": "e_[alpha/Fe]_upper",
    },
    inplace=True,
)

In [ ]:
# add metallicities, alpha and log g from high-res spectroscopic surveys

## Apogee
apogee = Table.read(get_abspath() + "data/external/apogee.fits", hdu=1)
# get relevant columns
apogee = apogee[
    [
        "GAIAEDR3_SOURCE_ID",
        "FE_H",
        "FE_H_ERR",
        "ALPHA_M",
        "ALPHA_M_ERR",
        "LOGG",
        "LOGG_ERR",
        "ASPCAPFLAGS",
    ]
]
apogee_df = apogee.to_pandas()
# qulaity cuts (drop entries where ALPHA_M or M_H is flagged)
apogee_df = apogee_df[
    ~apogee_df["ASPCAPFLAGS"].astype(str).str.contains("ALPHA_M|M_H|STAR_WARN")
].drop(columns="ASPCAPFLAGS")

apogee_df = apogee_df.rename(
    columns={
        "GAIAEDR3_SOURCE_ID": "gaiaID_DR3",
        "FE_H": "[Fe/H]_apogee",
        "FE_H_ERR": "e_[Fe/H]_apogee",
        "ALPHA_M": "[alpha/M]_apogee",
        "ALPHA_M_ERR": "e_[alpha/M]_apogee",
        "LOGG": "logg_apogee",
        "LOGG_ERR": "e_logg_apogee",
    }
)
# merge into targets
targets = pd.merge(
    targets,
    apogee_df.drop_duplicates(subset="gaiaID_DR3", keep="first"),
    on="gaiaID_DR3",
    how="left",
)

## GALAH
with warnings.catch_warnings():
    # silence warning for log(cm.s**-2) units
    warnings.simplefilter("ignore", UnitsWarning)
    galah = Table.read(get_abspath() + "data/external/galah.fits")
# quality cuts
galah = galah[galah["flag_sp"] == 0]
galah = galah[galah["flag_fe_h"] == 0]
galah = galah[galah["flag_alpha_fe"] == 0]
# get relevant columns
galah = galah[
    [
        "dr3_source_id",
        "fe_h",
        "e_fe_h",
        "alpha_fe",
        "e_alpha_fe",
        "logg",
        "e_logg",
    ]
]
galah_df = galah.to_pandas()
galah_df = galah_df.rename(
    columns={
        "dr3_source_id": "gaiaID_DR3",
        "fe_h": "[Fe/H]_galah",
        "e_fe_h": "e_[Fe/H]_galah",
        "alpha_fe": "[alpha/Fe]_galah",
        "e_alpha_fe": "e_[alpha/Fe]_galah",
        "logg": "logg_galah",
        "e_logg": "e_logg_galah",
    }
)
# merge into targets
targets = pd.merge(
    targets,
    galah_df.drop_duplicates(subset="gaiaID_DR3", keep="first"),
    on="gaiaID_DR3",
    how="left",
)

In [ ]:
# process asPIC
asPIC = Table.read(get_abspath() + f"data/external/asPIC_1.1.fits")
asPIC = asPIC[
    [
        "sourceId",
        "GLON",
        "GLAT",
        "gaiaV",
        "egaiaV",
        "Gmag",
        "eGmag",
        "Radius",
        "eRadius",
        "Mass",
        "eMass",
        "Teff",
        "eTeff",
        "sourceFlag",
    ]
]
for col in asPIC.colnames:
    asPIC[col] = asPIC[col][:, 0]
asPIC = asPIC.to_pandas()

# rename source flag
asPIC.rename(
    columns={
        "sourceId": "gaiaID_DR2",
        "sourceFlag": "Stellar Type",
        "egaiaV": "e_gaiaV",
        "eGmag": "e_Gmag",
        "eRadius": "e_Radius",
        "eMass": "e_Mass",
        "eTeff": "e_Teff",
    },
    inplace=True,
)
asPIC["Stellar Type"] = asPIC["Stellar Type"].map(
    {
        1: "FGK",  # FGK
        5: "FGK",  # FGK and known planet host
        2: "M",  # M
        6: "M",  # M and known planet host
    }
)

# match asPIC and targets on sourceId
data = pd.merge(targets, asPIC, on="gaiaID_DR2", how="inner")

In [ ]:
# add quadratic limb darkening coefficients for targets
ldcs = pd.read_csv(
    get_abspath() + "data/external/PLATO_NCAM_ldcs.csv",
    comment="#",
    usecols=["Teff", "logg", "u1", "u2"],
)

# use KNN to easily identify the nearest neighbour
knn = KNeighborsRegressor(n_neighbors=1)
knn.fit(ldcs[["Teff", "logg"]], ldcs[["u1", "u2"]])

data = data.assign(u1=np.nan, u2=np.nan)
data.loc[data[["Teff", "logg"]].dropna().index, ["u1", "u2"]] = knn.predict(
    data[["Teff", "logg"]].dropna()
)

In [ ]:
# sort columns
column_names = [
    "gaiaID_DR2",
    "gaiaID_DR3",
    "parallax",
    "e_parallax",
    "ra",
    "e_ra",
    "dec",
    "e_dec",
    "pmra",
    "e_pmra",
    "pmdec",
    "e_pmdec",
    "radial_velocity",
    "e_radial_velocity",
    "[alpha/Fe]",
    "e_[alpha/Fe]_lower",
    "e_[alpha/Fe]_upper",
    "[Fe/H]",
    "[Fe/H]_source",
    "e_[Fe/H]_lower",
    "e_[Fe/H]_upper",
    "logg",
    "logg_source",
    "e_logg_lower",
    "e_logg_upper",
    "[Fe/H]_apogee",
    "e_[Fe/H]_apogee",
    "[alpha/M]_apogee",
    "e_[alpha/M]_apogee",
    "logg_apogee",
    "e_logg_apogee",
    "[Fe/H]_galah",
    "e_[Fe/H]_galah",
    "[alpha/Fe]_galah",
    "e_[alpha/Fe]_galah",
    "logg_galah",
    "e_logg_galah",
    "GLON",
    "GLAT",
    "U",
    "V",
    "W",
    "UW",
    "R",
    "Z",
    "gaiaV",
    "e_gaiaV",
    "Gmag",
    "e_Gmag",
    "Radius",
    "e_Radius",
    "Mass",
    "e_Mass",
    "Teff",
    "e_Teff",
    "Stellar Type",
    "u1",
    "u2",
    "TD/D",
    "TD/H",
    "Population",
]
data = data[column_names]

In [ ]:
# save
data.to_csv(get_abspath() + "data/processed/all_sky_targets.csv", index=False)